In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as si
import yfinance as yf
import os

In [2]:
TSLA = yf.download("TSLA", start="2021-04-26", end="2022-04-26")

[*********************100%***********************]  1 of 1 completed


In [3]:
S= TSLA ['Adj Close'][-1]
print ('The Spot Price is $',round (S,2),'.')

The Spot Price is $ 998.02 .


In [4]:
log_return = np.log(TSLA['Adj Close'] / TSLA['Adj Close'].shift(1))
vol_h = np.sqrt(252) * log_return.std()
print('The annualised volatility is', round(vol_h*100,2), '%')

The annualised volatility is 53.74 %


In [5]:
VIX= yf.download('^VIX', start='2021-04-26', end='2022-04-26')

[*********************100%***********************]  1 of 1 completed


In [9]:
VIX= pd.DataFrame(VIX)
VIX

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-04-26,17.940001,18.170000,16.870001,17.639999,17.639999,0
2021-04-27,17.620001,18.160000,16.969999,17.559999,17.559999,0
2021-04-28,17.469999,17.840000,16.670000,17.280001,17.280001,0
2021-04-29,16.879999,18.870001,16.770000,17.610001,17.610001,0
2021-04-30,17.670000,19.250000,17.639999,18.610001,18.610001,0
...,...,...,...,...,...,...
2022-04-19,22.549999,22.920000,20.360001,21.370001,21.370001,0
2022-04-20,21.129999,21.320000,19.750000,20.320000,20.320000,0
2022-04-21,20.240000,23.280001,19.809999,22.680000,22.680000,0


In [10]:
V= VIX['Adj Close'][-1]
print (V)

27.020000457763672


In [ ]:
type (VIX)

In [19]:
def newton_vol_call(S, K, T, C, r):
    
    #S: spot price
    #K: strike price
    #T: time to maturity
    #C: Call value
    #r: risk free rate
    #sigma: volatility of underlying asset
   
    MAX_ITERATIONS = 100
    tolerance = 0.000001
    
    sigma = 0.25
    
    for i in range(0, MAX_ITERATIONS):
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        price = S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0)
        vega = S * np.sqrt(T) * si.norm.pdf(d1, 0.0, 1.0)

        diff = C - price

        if (abs(diff) < tolerance):
            return sigma
        else: 
            sigma = sigma + diff/vega
        
        print(i,sigma,diff)
        
    return sigma

In [27]:
TESLA = yf.Ticker("TSLA")
opt = TESLA.option_chain('2022-06-03')
opt.calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,TSLA220603C00750000,2022-04-22 17:44:55+00:00,750.0,267.38,175.55,184.65,0.00,0.000000,2.0,1,0.778109,True,REGULAR,USD
1,TSLA220603C00780000,2022-04-21 13:56:52+00:00,780.0,307.94,151.15,159.85,0.00,0.000000,NaN,1,0.738955,True,REGULAR,USD
2,TSLA220603C00800000,2022-04-26 17:02:17+00:00,800.0,138.00,133.65,141.05,0.00,0.000000,13.0,15,0.688754,True,REGULAR,USD
3,TSLA220603C00820000,2022-04-25 14:00:45+00:00,820.0,193.96,122.00,129.75,0.00,0.000000,4.0,5,0.703204,True,REGULAR,USD
4,TSLA220603C00840000,2022-04-27 14:06:33+00:00,840.0,105.00,108.45,115.30,-88.03,-45.604310,1.0,1,0.684482,True,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,TSLA220603C01600000,2022-04-27 14:45:40+00:00,1600.0,0.43,0.06,0.80,-0.07,-13.999998,2.0,48,0.690677,False,REGULAR,USD
67,TSLA220603C01625000,2022-04-26 17:11:58+00:00,1625.0,1.27,0.00,0.76,0.00,0.000000,1.0,2,0.697269,False,REGULAR,USD
68,TSLA220603C01650000,2022-04-27 13:45:13+00:00,1650.0,0.70,0.01,0.70,0.31,79.487180,1.0,328,0.708011,False,REGULAR,USD
69,TSLA220603C01700000,2022-04-27 15:54:46+00:00,1700.0,0.18,0.00,0.46,-0.20,-52.631577,7.0,56,0.706546,False,REGULAR,USD


In [28]:
impvol = newton_vol_call(S, 890.00, 6/52, float(opt.calls.lastPrice[opt.calls.strike == 890.00]), 0.0192)
print(impvol*100)

0 -0.4180301655943116 -33.084136726394945
1 0.6334940032718551 95.42363901718753
2 0.018320720919685174 -67.34612410344653
3 -6.587689089157626e+75 -29.98952942246808
4 inf 968.0304901087819
5 nan nan
6 nan nan
7 nan nan
8 nan nan
9 nan nan
10 nan nan
11 nan nan
12 nan nan
13 nan nan
14 nan nan
15 nan nan
16 nan nan
17 nan nan
18 nan nan
19 nan nan
20 nan nan
21 nan nan
22 nan nan
23 nan nan
24 nan nan
25 nan nan
26 nan nan
27 nan nan
28 nan nan
29 nan nan
30 nan nan
31 nan nan
32 nan nan
33 nan nan
34 nan nan
35 nan nan
36 nan nan
37 nan nan
38 nan nan
39 nan nan
40 nan nan
41 nan nan
42 nan nan
43 nan nan
44 nan nan
45 nan nan
46 nan nan
47 nan nan
48 nan nan
49 nan nan
50 nan nan
51 nan nan
52 nan nan
53 nan nan
54 nan nan
55 nan nan
56 nan nan
57 nan nan
58 nan nan
59 nan nan
60 nan nan
61 nan nan
62 nan nan
63 nan nan
64 nan nan
65 nan nan
66 nan nan
67 nan nan
68 nan nan
69 nan nan
70 nan nan
71 nan nan
72 nan nan
73 nan nan
74 nan nan
75 nan nan
76 nan nan
77 nan nan
78 nan nan


C:\Users\PRIYAN~1.KOT\AppData\Local\Temp/ipykernel_8996/997879902.py:26: RuntimeWarning: divide by zero encountered in double_scalars
  sigma = sigma + diff/vega
C:\Users\PRIYAN~1.KOT\AppData\Local\Temp/ipykernel_8996/997879902.py:16: RuntimeWarning: invalid value encountered in double_scalars
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
C:\Users\PRIYAN~1.KOT\AppData\Local\Temp/ipykernel_8996/997879902.py:17: RuntimeWarning: invalid value encountered in double_scalars
  d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))


In [23]:
float(opt.calls.lastPrice[opt.calls.strike == 890.00])

71.45

In [24]:
import mibian

In [26]:
c = mibian.BS([S, 890.00, 1.92, 39], callPrice = float(opt.calls.lastPrice[opt.calls.strike == 890.00]))
c.impliedVolatility
print('The implied volatility is', round(c.impliedVolatility,2) , '% for the one-month call with strike $ 890.00' ) 

The implied volatility is 0.0 % for the one-month call with strike $ 890.00
